A program that reads and processes images for a Convolutional Neural Network (CNN) to classify as images as good or bad.

In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras import optimizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.models import load_model

In [ ]:
df = pd.read_csv('')

In [ ]:
def read_images(paths): 
    """
    Reads in all images and returns list of picture id numbers based on the image name
    
    Parameters
    ----------
    paths : string
    
    Returns
    ----------
    images and list of id numbers
    """
    #Get list of images
    images = (glob(paths + '*.jpg'))
    #Read images from list
    data = [cv2.imread(file) for file in images]
    #Separates name of image from path
    names = [os.path.split(file)[1] for file in images]
    #Separates first part of image name from .jpg to get id number
    id_num = [int(file.split('.')[0]) for file in names]
    
    return images, id_num

In [ ]:
def cnn_layers(input_size):
    """
    Builds layers of Convolutional Neural Net
    
    Parameters
    -----------
    input_size = tuple
    
    Returns
    -----------
    model
    """
    model = Sequential()
        
    #first convolutional layer and pooling
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(input_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #second convolutional layer and pooling
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, border_mode='valid', input_shape=(input_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #flatten images
    model.add(Flatten())
    
    #first dense layer
    model.add(Dense(2048, init='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    #second dense layer
    model.add(Dense(2048, init='glorot_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    return model  

In [ ]:
def implement_cnn(model, x_train, y_train, x_test, y_test, batch_size = 4, nb_classes = 2, nb_epoch = 40)
    """
    Fits model to the data
    
    Parameters
    ------------
    x_train = array
    x_test = array
    y_train = data frame or array
    y_test = data frame or array
    batch_size = integer
    nb_classes = integer
    nb_epoch = integer
    
    Returns
    ------------
    model metrics
    """
    #initializes optimizer SGD
    #need to see which learning rate (lr) achieves best results
    sgd = optimizers.SGD(lr=0.005, decay=le-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    #early stopping batch = x_train, y_train, x_test, y_test
    #need to experiment with patience 
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
    
    checkpointer = ModelCheckpoint(filepath=('checkpoint.hdf5'), verbose=1, save_best+only=True)
    
    hist = callbacks.History()
    
    model.fit(x_train, y_train, verbose=2, callbacks = [early_stopping, checkpointer, hist], batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(x_test, y_test), steps_per_epoch=10)
    
    return model.evaluate(x_train, y_train, verbose=2)